In [1]:
%load_ext autoreload

# --------------- #
# region: Imports #
import os
import sys
module_path = os.path.abspath('../../..')
if module_path not in sys.path:
    sys.path.insert(0, module_path)
# endregion       #
# --------------- #

In [2]:
# %autoreload 2
# from examples.RPG.test import train_transformer_model
# from examples.RPG.utils import load_config
# import argparse

# cfg = load_config(argparse.Namespace(config='../configs/config.yaml'))

# train_transformer_model(cfg, 'iRainbowModel_20230916-17091694900425')


In [3]:
# import argparse
# from gem.models.transformer import ViTOneHot
# from gem.config import load_config

# cfg = load_config(argparse.Namespace(config='../configs/transformer.yaml'))

# model = ViTOneHot(**vars(cfg.model))